In [14]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import sqlite3
import math
import os
import json

from matplotlib import image
from flask import g, Flask, render_template, request, session, redirect
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, jsonify
from PIL import Image

from werkzeug.serving import run_simple

In [15]:
objects = json.load(open("objects_nlp.json"))

In [16]:
con = sqlite3.connect('rijksstudio.db')

In [17]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS Clicks
               (painting_id INTEGER, 
                object_name TEXT, 
                user_id INTEGER,
                x INTEGER, 
                y INTEGER
                )''')

cur.execute("""CREATE TABLE IF NOT EXISTS Users
               (user_id INTEGER PRIMARY KEY,
                username TEXT,
                password TEXT
                )""")
        
# Insert 5000 spoof rows of click data
# for _ in range(5000):
#     painting_id = np.random.choice(os.listdir("static/images/paintings/"))
#     object_name = np.random.choice(["dog", "human", "cow", "cat", "egg"])
#     user_id = np.random.randint(0, 10000)
#     x = np.random.randint(0, 500)
#     y = np.random.randint(0, 500)
    
#     cur.execute(f"INSERT INTO Clicks VALUES ('{painting_id}', '{object_name}', '{user_id}', {x}, {y})")

# Save the changes
con.commit()
con.close()

In [18]:
# Connect to the database and setup the app
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

def get_db():
    # Connect to the database
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect("rijksstudio.db")
    return db

@app.teardown_appcontext
def close_connection(exception):
    # Close the database
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

In [19]:
@app.route('/')
def index():
    # If logged in, show a painting
    if "user_id" in session:
        # Choose random painting, but make sure there are objects in it
        items = []
        while not items:
            painting = np.random.choice(os.listdir("static/images/paintings/"))
            items = objects[painting.split(".")[0]]
        
        obj = np.random.choice(items).replace(" ", "_")
        
        session['painting'] = f"static/images/paintings/{painting}"
        session["object"] = obj
        
        # Read image for dimensions
        dims = Image.open(session['painting']).size
                
        # Show the image on an html page
        return render_template('test.html', 
                               name = obj, 
                               url = f'static/images/paintings/{painting}', 
                               width = dims[0]/2, 
                               height = dims[1]/2)
    else: # Else, redirect the user to the login page
        return redirect("/login")

In [20]:
@app.route("/register", methods=["GET", "POST"])
def register():
    # If getting the page, show the html
    if request.method == "GET":
        return render_template("register.html")
    else: # if posting the page, store the data
        username = request.form.get("username")
        password = request.form.get("password")
        
        cur = get_db().cursor()   
        q = cur.execute(f"""INSERT INTO Users
                           VALUES (null, '{username}', '{password}')
                           """)
        # Commit changes
        get_db().commit()

        return redirect("/login")

In [21]:
@app.route("/login", methods=["GET", "POST"])
def login():
    """Compare a username and password against the database and let the user log in if both are correct."""

    # forget any user_id
    session.clear()
    cur = get_db().cursor()
    
    # If posting, check input and attempt login
    if request.method == "POST":
        if not request.form.get("username") or not request.form.get("password"):
            return "please fill in everything"
                
        # Query for the user logging in 
        row = cur.execute(f"""SELECT user_id, password 
                               FROM Users 
                               WHERE username = '{request.form.get('username')}'""").fetchone()
            
        # If user not found or password does not match, throw an error
        if not row or not request.form.get("password") == row[1]:
            return "password error"

        # remember which user has logged in
        session["user_id"] = row[0]
        
        # redirect user to home page
        return redirect("/")

    else: # If getting, display html
        return render_template("login.html")

In [22]:
@app.route('/score', methods=["GET", "POST"])
def score():
    # Find where the user clicked
    coordinates = request.args.get('coords')
    print(coordinates)
    return coordinates
    x, y = map(int, coordinates.split(","))

    # Load in the image
    img = np.array(Image.open(session['painting']))
    
    dims = Image.open(session['painting']).size
    
    # Query the database for relevant clicks
    cur = get_db().cursor()   

    cur.execute(f"""INSERT INTO Clicks
                    VALUES ("{session["painting"].split("/")[-1]}", 
                            "{session["object"]}", 
                            "{session["user_id"]}", 
                            {x}, 
                            {y})""")
    
    get_db().commit()
    
    q = cur.execute(f"""SELECT x, y
                        FROM Clicks 
                        WHERE painting_id == '{session["painting"].split("/")[-1]}'
                        AND object_name == '{session["object"]}'""")   
    
    # Create border edges
    points = [(r[0], r[1]) for r in q]
    
    print(len(points))

    # Figure settings
    plt.figure(figsize = (16, 16))
    ax = plt.gca()
    
    if len(points) > 2:   
        if len(points) > 10:
            points = normalize(points)
        
        min_x, min_y, max_x, max_y = find_edges(points)

        v_lines = [min_x, max_x]
        h_lines = [min_y, max_y]
        box = (min_x, max_x, min_y, max_y)

        # Plot border
        for line in v_lines:
            plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                       ls='-', lw=1.5)

        for line in h_lines:
            plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                       ls='-', lw=1.5)
            
        plt.scatter([i[0] for i in points], [i[1] for i in points], c="b", s=10)
            
        final_score = calculate_score((x, y), box, 0)
    else: # We had no data yet on this painting/object, so we assume the user was right
        final_score = 100

    # Plot where the user clicked
    plt.scatter(x, y, c="r", s=10)
    plt.axis('off')
    plt.imshow(img)

    # Store the rendered figure
    plt.savefig(f'static/images/{session["painting"].split("/")[-1]}_{session["object"]}.png',
                bbox_inches='tight',
                dpi = "figure") # set to like 500 if we want very pretty high quality paintings that take forever to load

    plt.close()

    return render_template('score.html', 
                           name = session["object"], 
                           points = final_score, 
                           url = f'static/images/{session["painting"].split("/")[-1]}_{session["object"]}.png', 
                           width = dims[0]/2, 
                           height = dims[1]/2)

In [23]:
def normalize(points):
    # Convert clicks to array
    a = np.array(points)    
    # Calculate mean and standard deviation
    mean, stdev = np.mean(a, axis = 0), np.std(a, axis=0)
    
    # Normalize values, and see if they are more than a quarter standard deviation 
    # apart from the mean
    outliers = ((np.abs(a[:,0] - mean[0]) > 0.25 * stdev[0])
               * (np.abs(a[:,1] - mean[1]) > 0.25 * stdev[1]))
        
    return a[~outliers]

In [24]:
def find_edges(points):
    """Find the edges of the user clicks to determine the boxes"""
    min_x = min(points, key = lambda x: x[0])[0]
    min_y = min(points, key = lambda x: x[1])[1]

    max_x = max(points, key = lambda x: x[0])[0]
    max_y = max(points, key = lambda x: x[1])[1]
    
    return min_x, min_y, max_x, max_y

In [25]:
def calculate_score(click, box, time):
    
    # max score is 100
    score = 100
    
    # score goes down by 25 per second
    score -= time * 2.5
    
    # split click into x and y coordinates
    x, y = click
       
    # extract box edges
    min_x, max_x, min_y, max_y = box
    
    # find the biggest box size
    size = max(max_x - min_x, max_y - min_y)
   
    # If the click is within the box, give full points
    if (min_x < x < max_x) and (min_y < y < max_y):
        return score
    else:
        # distance to closest point on x-axis
        dx = max(min_x - x, 0, x - max_x)
        # distance to closest point on y-axis
        dy = max(min_y - y, 0, y - max_y)
        # pythagoras theorem
        distance = np.sqrt(dx**2 + dy**2)
                
        if distance > size:
            return 0
        else:
            return math.exp(-4 * (distance / size)) * score

In [ ]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2022 16:27:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 16:27:51] "GET /static/images/paintings/RP-P-H-1086.png HTTP/1.1" 200 -
